In [ ]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

def get_team_matches(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, features="lxml")
    matches = pd.read_html(response.content)[1]
    h1 = soup.find("h1").text
    team_name = re.search("[0-9]{4}-[0-9]{4} (.*) Stats", h1).group(1)
    matches["Team"] = team_name

    return matches, soup

def get_football_matches():

    # this season
    url = f"https://fbref.com/en/squads/{TEAM_ID}"
    matches, soup = get_team_matches(url)

    # prev season
    endpoint = soup.find("div", attrs={"class": "prevnext"}).find("a").attrs["href"]
    url_prev = f"https://fbref.com" + endpoint
    matches_prev, _ = get_team_matches(url_prev)